In [2]:
import pandas as pd
import numpy as np

In [26]:
df = pd.read_pickle("11_18_21.pkl")
df

,Code,Name,Record,Status,Lat,Lon,Wind,Pressure,DateTime,34kt_radius,50kt_radius,64kt_radius,Bearing,Distance,Speed,Rapid_NHC6,Rapid_NHC12,Rapid_NHC18,Rapid_NHC24,Accel
0,AL011851,UNNAMED,,HU,28.0,-94.8,80,-999,1851-06-25 00:00:00,-999.0,-999.0,-999.0,0.000000,0.000000,0.000000,False,False,False,False,0.000000
1,AL011851,UNNAMED,,HU,28.0,-95.4,80,-999,1851-06-25 06:00:00,-999.0,-999.0,-999.0,-89.859158,36.671593,6.111932,False,False,False,False,0.000000
2,AL011851,UNNAMED,,HU,28.0,-96.0,80,-999,1851-06-25 12:00:00,-999.0,-999.0,-999.0,-89.859158,36.671593,6.111932,False,False,False,False,0.000000
3,AL011851,UNNAMED,,HU,28.1,-96.5,80,-999,1851-06-25 18:00:00,-999.0,-999.0,-999.0,-77.178366,31.312111,5.218685,False,False,False,False,0.000000
4,AL011851,UNNAMED,L,HU,28.2,-96.8,80,-999,1851-06-25 21:00:00,-999.0,-999.0,-999.0,-69.318979,19.562422,6.520807,False,False,False,False,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34599,AL312020,IOTA,,HU,13.7,-84.7,75,965,2020-11-17 12:00:00,122.5,37.5,12.5,-89.893421,60.493955,10.082326,False,False,False,False,-5.833333
34600,AL312020,IOTA,,TS,13.7,-85.7,55,988,2020-11-17 18:00:00,90.0,15.0,0.0,-89.881578,67.215497,11.202583,False,False,False,False,-3.333333
34601,AL312020,IOTA,,TS,13.8,-86.7,40,1000,2020-11-18 00:00:00,67.5,0.0,0.0,-84.040273,67.551977,11.258663,False,False,False,False,-2.500000
34602,AL312020,IOTA,,TS,13.8,-87.8,35,1005,2020-11-18 06:00:00,35.0,0.0,0.0,-89.868803,73.905664,12.317611,False,False,False,False,-0.833333


In [49]:
def calculate_category(wind):
    cat_breaks = [64,83,96,113,137,300]
    for i,cat in enumerate(cat_breaks):
        if wind < cat:
            return i
    return np.nan

### HURDAT Record Identifiers

These values provide additional information about a specific observation from a storm. The one we are most interested in is the 'L' identifier for Landfall.

While prediciting RI events anywhere is useful in theory, **we are most concerned with predicting RI when the storm is close to land.**

C – Closest approach to a coast, not followed by a landfall  
G – Genesis  
I – An intensity peak in terms of both pressure and wind  
L – Landfall (center of system crossing a coastline)  
P – Minimum in central pressure  
R – Provides additional detail on the intensity of the cyclone when rapid changes are underway  
S – Change of status of the system  
T – Provides additional detail on the track (position) of the cyclone     
W – Maximum sustained wind spee

In [56]:
df_all = pd.DataFrame()
codes = df.Code.unique()
for code in codes:
    df_strom = df[df.Code == code]
    df_all = df_all.append({
        'Code'         : code,
        'Name'         : df_strom.iloc[0].Name,
        'Start_Date'   : df_strom.DateTime.min(),
        'End_Date'     : df_strom.DateTime.max(),
        'Min_Lat'      : df_strom.Lat.max(),
        'Max_Lat'      : df_strom.Lat.max(),
        'Min_Lon'      : df_strom.Lon.max(),
        'Max_Lon'      : df_strom.Lon.max(),
        'Max_Wind'     : df_strom.Wind.max(),
        'Min_Pressure' : df_strom.Pressure.max(),
        'Max_Accel'    : df_strom.Accel.max(),
        'Category' : calculate_category(df_strom.Wind.max()),
        'Max_34kt'     : df_strom['34kt_radius'].max(),
        'Max_50kt'     : df_strom['50kt_radius'].max(),
        'Max_64kt'     : df_strom['64kt_radius'].max(),
        'Rapid_NHC6'   : df_strom.Rapid_NHC6.any(),
        'Rapid_NHC12'  : df_strom.Rapid_NHC12.any(),
        'Rapid_NHC18'  : df_strom.Rapid_NHC18.any(),
        'Rapid_NHC24'  : df_strom.Rapid_NHC24.any(),
    }, ignore_index=True)

In [54]:
df_all.Rapid_NHC24.sum() / len(df_all)

0.3343782654127482

In [55]:
df_all

,Code,Name,Start_Date,End_Date,Min_Lat,Max_Lat,Min_Lon,Max_Lon,Max_Wind,Min_Pressure,Max_Accel,Max_Category,Max_34kt,Max_50kt,Max_64kt,Rapid_NHC6,Rapid_NHC12,Rapid_NHC18,Rapid_NHC24
0,AL011851,UNNAMED,1851-06-25 00:00:00,1851-06-28 00:00:00,31.0,31.0,-94.8,-94.8,80.0,-999.0,0.000000,1.0,-999.0,-999.0,-999.00,0.0,0.0,0.0,0.0
1,AL021851,UNNAMED,1851-07-05 12:00:00,1851-07-05 12:00:00,22.2,22.2,-97.6,-97.6,80.0,-999.0,0.000000,1.0,-999.0,-999.0,-999.00,0.0,0.0,0.0,0.0
2,AL041851,UNNAMED,1851-08-16 00:00:00,1851-08-27 18:00:00,48.5,48.5,-48.0,-48.0,100.0,-999.0,1.666667,3.0,-999.0,-999.0,-999.00,0.0,0.0,0.0,0.0
3,AL011852,UNNAMED,1852-08-19 00:00:00,1852-08-30 00:00:00,41.0,41.0,-67.1,-67.1,100.0,961.0,1.666667,3.0,-999.0,-999.0,-999.00,0.0,0.0,0.0,0.0
4,AL021852,UNNAMED,1852-09-05 00:00:00,1852-09-06 18:00:00,19.7,19.7,-64.1,-64.1,70.0,-999.0,0.000000,1.0,-999.0,-999.0,-999.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,AL262020,DELTA,2020-10-04 18:00:00,2020-10-11 18:00:00,35.1,35.1,-76.2,-76.2,120.0,1007.0,4.166667,4.0,117.5,45.0,28.75,0.0,1.0,1.0,1.0
953,AL272020,EPSILON,2020-10-16 12:00:00,2020-10-26 12:00:00,52.1,52.1,-28.5,-28.5,100.0,1013.0,2.500000,3.0,292.5,127.5,57.50,0.0,0.0,1.0,1.0
954,AL282020,ZETA,2020-10-24 12:00:00,2020-10-30 00:00:00,39.5,39.5,-71.7,-71.7,100.0,1007.0,2.500000,3.0,185.0,50.0,20.00,0.0,0.0,1.0,1.0
955,AL292020,ETA,2020-10-31 18:00:00,2020-11-14 00:00:00,35.7,35.7,-69.2,-69.2,130.0,1006.0,4.166667,4.0,122.5,35.0,20.00,0.0,1.0,1.0,1.0
